In [1]:
#PySpark Dynamodb fetch "FindMyMovie" personality records.
try:
    spark.stop()
except:
    print("No Session")
#Starting PySpark Session
from pyspark.sql import SparkSession

spark=SparkSession.builder.appName('data_processing')\
    .master("spark://192.168.1.100:7077") \
    .config('spark.executor.memory','4600m') \
    .getOrCreate()   

#Required imports other than SPARK main
import csv
import json
from pyspark.sql.functions import udf
from pyspark import SparkContext
from pyspark.sql.types import StringType ,IntegerType

In [160]:
#Pull Date Dec 2 1030 AM

#Dynamodb Handeler to connect to "FindMyMovie" table.
import boto3
from boto3.dynamodb.conditions import Key, Attr
from botocore.exceptions import ClientError
import botocore


class   dynamodb_FindMyMovies():
    def __init__(self):
        # Get the service resource.
        self.dynamodb = ''#boto3.resource('dynamodb',endpoint_url='http://localhost:8000')
        self.table = ''#dynamodb.Table('FindMyMovies_Table')
        

    def check_state(self):
        try:
            self.dynamodb= boto3.resource('dynamodb',region_name='us-east-1',aws_access_key_id="Your Keys",aws_secret_access_key="Your keys")
            self.table= self.dynamodb.Table('Your Table')
            print(self.table.creation_date_time)
            return 0
        
        except botocore.exceptions.ParamValidationError as e:
            print(e)
        except:
            print("Couldn't connect with Database or Table")
            return 1
        

    def table_scan_register(self):
        self.check_state()
        response = self.table.scan(FilterExpression=Attr('register_date').gt('2019-1-1'))
        return response
    
    def table_scan_combined_scores(self):
        self.check_state()
        response = self.table.scan(FilterExpression=Attr('r_type').eq('ipip120_response'))
        return response
                              
    def table_scan_movies(self):
        self.check_state()
        response = self.table.scan(IndexName='titleID-LSI',FilterExpression=Attr('rating').gt(-1))
        return response        
        

In [ ]:
#Fetch New users with register date
response_scan=dynamodb_FindMyMovies()
e = response_scan.table_scan_register()

dictionary = e['Items']

#popping r_type , i-e metadata.
for item in dictionary:
    item.pop('r_type', None)
    
#saving metadata to csv.
csv_columns = ['userID','full_name','email','country','gender','register_date','birthdate','age','ip_addr']
dict_data = dictionary
csv_file = "/root/datasets/FYP/metadata.csv"
try:
    with open(csv_file, 'w') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=csv_columns)
        writer.writeheader()
        for data in dict_data:
            writer.writerow(data)
except IOError:
    print("I/O error")


In [167]:
#Read metadata csv and get userID and email column to get reference csv
df=spark.read.csv('/root/datasets/FYP/metadata.csv',inferSchema=True,header=True)

#Keeping only required columns to reference email and userID
columns_to_drop = ['full_name', 'country','gender','register_date','birthdate','age','ip_addr']
df = df.drop(*columns_to_drop)

df.columns
df.head(2)

#saving reference file
#df.write.format('csv').option('header',True).mode('overwrite').option('sep',',').save('file:///home/tangr/output.csv')
df.write.format('csv').option('header',True).option('sep',',').save('/root/datasets/FYP/email_userID_ref.csv')

In [137]:
# Fetching and converting email and combined scores to csv

try:
    del e
except:
    pass

response_scan=dynamodb_FindMyMovies()
e = response_scan.table_scan_combined_scores()
e = e['Items']


li = []
new={}
for x in e:
    if x['email']:
        if x['r_type'] == 'ipip120_response': 
            #Appending entries
            new['email'] = x['email']
            new['EXT'] = int(x['EXT'])
            new['AGG'] = int(x['AGG'])
            new['CON'] = int(x['CON'])
            new['NEU'] = int(x['NEU'])
            new['OPN'] = int(x['OPN'])
            li.append(new.copy())

            


csv_columns = ['email','EXT','AGG','CON','NEU','OPN']
dict_data = li
csv_file = "/root/datasets/FYP/combined_scores.csv"
try:
    with open(csv_file, 'w') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=csv_columns)
        writer.writeheader()
        for data in dict_data:
            writer.writerow(data)
except IOError:
    print("I/O error")


            

2020-10-15 15:02:35.566000-04:00


In [161]:
#Scaning LSI to get movies and ratings
response_scan=dynamodb_FindMyMovies()
e = response_scan.table_scan_movies()
e = e['Items']


csv_columns = ['email','titleID','rating','r_type']
dict_data = e
csv_file = "/root/datasets/FYP/ratings.csv"
try:
    with open(csv_file, 'w') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=csv_columns)
        writer.writeheader()
        for data in dict_data:
            writer.writerow(data)
except IOError:
    print("I/O error")


2020-10-15 15:02:35.566000-04:00


In [184]:
#Cleansing all files to replace emails with userIDs for privacy
#Read metadata csv and get userID and email column to get reference csv

#Cleaning metadata file
df_meta=spark.read.csv('/root/datasets/FYP/metadata.csv',inferSchema=True,header=True)

#Keeping only required columns to reference email and userID
columns_to_drop = ['full_name','birthdate','age','ip_addr','email']
df_meta = df_meta.drop(*columns_to_drop)


df_meta.columns
df_meta.head(2)

#saving reference file
#df.write.format('csv').option('header',True).mode('overwrite').option('sep',',').save('file:///home/tangr/output.csv')
df_meta.write.format('csv').option('header',True).option('sep',',').save('/root/datasets/FYP/meta.csv')




In [276]:
#Loading reference file and storing it as a python dict obj
with open('/root/datasets/FYP/email_userID_ref.csv', mode='r') as infile:
    reader = csv.reader(infile)
    with open('coors_new.csv', mode='w') as outfile:
        writer = csv.writer(outfile)
        mydict = {rows[1]:rows[0] for rows in reader}




In [42]:
from pyspark.sql.functions import udf
from pyspark import SparkContext
from pyspark.sql.types import StringType ,IntegerType

#Generic lookup function and register as udf
def fetch_userID(email):
        #a =df_ref.filter(df_ref['email']==email).collect()[0]
        try:
            return mydict[email]
        except:
            return 'default'
get_udf=udf(fetch_userID,StringType())

In [43]:
#Changing chaning combined scores files with transformation
#loading files to be transformed for privacy
df_scores=spark.read.csv('/root/datasets/FYP/combined_scores.csv',inferSchema=True,header=True)

new_df = df_scores.withColumn('email',get_udf(df_scores.email))
new_df = new_df.withColumnRenamed("email", "userID")
new_df.write.format('csv').option('header',True).option('sep',',').save('/root/datasets/FYP/O/ipip_scores')


In [44]:
new_df.show(5)

+-------+---+---+---+---+---+
| userID|EXT|AGG|CON|NEU|OPN|
+-------+---+---+---+---+---+
|default| 63| 81| 53| 70| 68|
|default| 35| 68| 60| 64| 70|
|default| 41| 58| 44| 88| 75|
|default| 51| 72| 73| 65| 74|
|default| 66| 62| 67| 66| 80|
+-------+---+---+---+---+---+
only showing top 5 rows



In [333]:
#Changing chaning combined scores files with transformation
#loading files to be transformed for privacy
df_scores=spark.read.csv('/root/datasets/FYP/ratings.csv',inferSchema=True,header=True)

new_df = df_scores.withColumn('email',get_udf(df_scores.email))
new_df = new_df.withColumnRenamed("email", "userID")
new_df.write.format('csv').option('header',True).option('sep',',').save('/root/datasets/FYP/O/ratings.csv')

In [334]:
new_df.show(5)

+--------------------+-------+------+--------------------+
|              userID|titleID|rating|              r_type|
+--------------------+-------+------+--------------------+
|69dedd10-2371-405...| 299534|     8|2020-11-22 20:43:...|
|fb3cc999-2cd2-466...|   1271|     7|2020-11-26 11:53:...|
|fb3cc999-2cd2-466...|   1372|     9|2020-11-26 11:51:...|
|fb3cc999-2cd2-466...|   1422|     9|2020-11-26 11:51:...|
|fb3cc999-2cd2-466...| 146216|     6|2020-11-26 11:52:...|
+--------------------+-------+------+--------------------+
only showing top 5 rows



In [402]:
#Statisical Analysis of plain scores
df_scores=spark.read.csv('/root/datasets/FYP/combined_scores.csv',inferSchema=True,header=True)

df_scores.agg({'EXT': 'mean'}).show()
df_scores.agg({'OPN': 'mean'}).show()
df_scores.agg({'AGG': 'mean'}).show()
df_scores.agg({'NEU': 'mean'}).show()
df_scores.agg({'OPN': 'mean'}).show()
df_scores.agg({'EXT': 'variance'}).show()
df_scores.agg({'OPN': 'variance'}).show()
df_scores.agg({'AGG': 'variance'}).show()
df_scores.agg({'NEU': 'variance'}).show()
df_scores.agg({'OPN': 'variance'}).show()

+-----------------+
|         avg(EXT)|
+-----------------+
|61.84782608695652|
+-----------------+

+-----------------+
|         avg(OPN)|
+-----------------+
|68.90217391304348|
+-----------------+

+-----------------+
|         avg(AGG)|
+-----------------+
|70.55434782608695|
+-----------------+

+-----------------+
|         avg(NEU)|
+-----------------+
|62.55434782608695|
+-----------------+

+-----------------+
|         avg(OPN)|
+-----------------+
|68.90217391304348|
+-----------------+

+-----------------+
|    variance(EXT)|
+-----------------+
|84.28428093645482|
+-----------------+

+-----------------+
|    variance(OPN)|
+-----------------+
|70.74856665074051|
+-----------------+

+-----------------+
|    variance(AGG)|
+-----------------+
|96.05195891065455|
+-----------------+

+-----------------+
|    variance(NEU)|
+-----------------+
|140.0959149546107|
+-----------------+

+-----------------+
|    variance(OPN)|
+-----------------+
|70.74856665074051|
+----------

In [461]:
#Importing impurity dataset and transforming it
df_merge=spark.read.csv('/root/datasets/personality-isf2018/personality-data.csv',inferSchema=True,header=True)
columns_to_drop = [' assigned metric',' assigned condition',' movie_1',
 ' predicted_rating_1',
 ' movie_2',
 ' predicted_rating_2',
 ' movie_3',
 ' predicted_rating_3',
 ' movie_4',
 ' predicted_rating_4',
 ' movie_5',
 ' predicted_rating_5',
 ' movie_6',
 ' predicted_rating_6',
 ' movie_7',
 ' predicted_rating_7',
 ' movie_8',
 ' predicted_rating_8',
 ' movie_9',
 ' predicted_rating_9',
 ' movie_10',
 ' predicted_rating_10',
 ' movie_11',
 ' predicted_rating_11',
 ' movie_12',
 ' predicted_rating_12',
 ' is_personalized',
 ' enjoy_watching ']
df_merge = df_merge.drop(*columns_to_drop)
df_merge = df_merge.withColumnRenamed("userid", "userID")
df_merge = df_merge.withColumnRenamed(" openness", "OPN")
df_merge = df_merge.withColumnRenamed(" agreeableness", "AGG")
df_merge = df_merge.withColumnRenamed(" emotional_stability", "EMO")
df_merge = df_merge.withColumnRenamed(" conscientiousness", "CON")
df_merge = df_merge.withColumnRenamed(" extraversion", "EXT")
df_merge.show(5)



+--------------------+---+---+---+---+---+
|              userID|OPN|AGG|EMO|CON|EXT|
+--------------------+---+---+---+---+---+
|8e7cebf9a234c064b...|5.0|2.0|3.0|2.5|6.5|
|77c7d756a093150d4...|7.0|4.0|6.0|5.5|4.0|
|b7e8a92987a530cc3...|4.0|3.0|4.5|2.0|2.5|
|92561f21446e017dd...|5.5|5.5|4.0|4.5|4.0|
|030001ac2145a938b...|5.5|5.5|3.5|4.5|2.5|
+--------------------+---+---+---+---+---+
only showing top 5 rows



In [462]:
#Reversing EMOTIONAL STABILY to standard NEUROTICISM model schema
def reverse(x):
        return int(8-x) 
rev=udf(reverse,StringType())
new_df = df_merge.withColumn('EMO',rev(df_merge.EMO))
df_merge = new_df.withColumnRenamed("EMO", "NEU")
df_merge.show(5)


+--------------------+---+---+---+---+---+
|              userID|OPN|AGG|NEU|CON|EXT|
+--------------------+---+---+---+---+---+
|8e7cebf9a234c064b...|5.0|2.0|  5|2.5|6.5|
|77c7d756a093150d4...|7.0|4.0|  2|5.5|4.0|
|b7e8a92987a530cc3...|4.0|3.0|  3|2.0|2.5|
|92561f21446e017dd...|5.5|5.5|  4|4.5|4.0|
|030001ac2145a938b...|5.5|5.5|  4|4.5|2.5|
+--------------------+---+---+---+---+---+
only showing top 5 rows



In [463]:
#Scailing all values from scale of 7 to scale of 100
def scale(x):
        return (x/7)*100
scale_100=udf(scale,IntegerType())
df_merge = df_merge.withColumn('OPN',scale(df_merge.OPN))
df_merge = df_merge.withColumn('EXT',scale(df_merge.EXT))
df_merge = df_merge.withColumn('AGG',scale(df_merge.AGG))
df_merge = df_merge.withColumn('NEU',scale(df_merge.NEU))
df_merge = df_merge.withColumn('CON',scale(df_merge.CON))

df_merge = df_merge.withColumn("OPN", df_merge.OPN.cast(IntegerType()))
df_merge = df_merge.withColumn("AGG", df_merge.AGG.cast(IntegerType()))
df_merge = df_merge.withColumn("NEU", df_merge.NEU.cast(IntegerType()))
df_merge = df_merge.withColumn("EXT", df_merge.EXT.cast(IntegerType()))
df_merge = df_merge.withColumn("CON", df_merge.CON.cast(IntegerType()))
df_merge.show(2)
df_merge.write.format('csv').option('header',True).option('sep',',').save('/root/datasets/FYP/O/imp_scores.csv')

In [490]:
#Merging original with impurity dataset
df_1=spark.read.csv('/root/datasets/FYP/O/ipip_120_scores.csv',inferSchema=True,header=True)
mega_df = df_1.union(df_merge)
mega_df.write.format('csv').option('header',True).option('sep',',').save('/root/datasets/FYP/O/agg/scores.csv')

In [498]:
df_1=spark.read.csv('/root/datasets/FYP/O/agg/all_scores.csv',inferSchema=True,header=True)
#Now adjust imported ratings scores

In [53]:
#Creating a lookup for movieID to TMDB movie ID.
#Loading reference file and storing it as a python dict obj
mydict={}
mydict2={}

with open('/root/datasets/ml-20m/links.csv', mode='r') as infile:
    reader = csv.reader(infile)
    with open('coors_new.csv', mode='w') as outfile:
        writer = csv.writer(outfile)
        mydict = {rows[0]:rows[2] for rows in reader}

with open('/root/datasets/ml-latest/links.csv', mode='r') as infile:
    reader = csv.reader(infile)
    with open('coors_new.csv', mode='w') as outfile:
        writer = csv.writer(outfile)
        mydict2 = {rows[0]:rows[2] for rows in reader}


1026765

In [58]:
df_mov=spark.read.csv('/root/datasets/personality-isf2018/ratings.csv',inferSchema=True,header=True)
df_mov = df_mov.withColumnRenamed("useri", "userID")
df_mov = df_mov.withColumnRenamed(" tstamp ", "r_type")
df_mov = df_mov.withColumnRenamed(" movie_id", "titleID")
df_mov = df_mov.withColumnRenamed(" rating", "rating")



df_mov.show(5)

+--------------------+-------+------+--------------------+
|              userID|titleID|rating|              r_type|
+--------------------+-------+------+--------------------+
|8e7cebf9a234c064b...|    1.0|   5.0| 2001-09-10 17:19...|
|8e7cebf9a234c064b...|    2.0|   4.0| 2001-09-28 11:34...|
|8e7cebf9a234c064b...|    3.0|   4.0| 2001-09-28 11:42...|
|8e7cebf9a234c064b...|    5.0|   5.0| 2001-09-28 11:27...|
|8e7cebf9a234c064b...|    6.0|   4.0| 2002-01-07 18:12...|
+--------------------+-------+------+--------------------+
only showing top 5 rows



In [59]:
df_mov = df_mov.withColumn("rating", df_mov["rating"].cast("integer"))
df_mov = df_mov.withColumn("titleID", df_mov["titleID"].cast("integer"))
df_mov = df_mov.withColumn("titleID", df_mov["titleID"].cast("string"))


In [67]:
df.filter("titleID is NULL").count()

0

In [60]:
#Converting titleIDs to TMDB Ids by writing UDF and searching in lookup dict

def tmdb_conv(x):
    a = mydict.get(x,None)
    if a is None:
        b = mydict2.get(x,None)
        return b
    else:
        return a 
tmdb_udf=udf(tmdb_conv,StringType())

df_new = df_mov.withColumn('titleID',tmdb_udf(df_mov.titleID))



In [63]:
df = df_new.filter(df_new.titleID. isNotNull())
df.count()

1026765

In [70]:
df= df.withColumn("titleID", df["titleID"].cast("integer"))
df = df.filter(df.titleID. isNotNull())
df.count()


1025835

In [76]:
new_DF.filter("titleID is NULL").show()

+------+-------+------+------+
|userID|titleID|rating|r_type|
+------+-------+------+------+
+------+-------+------+------+



In [72]:

def rating_conv(x):
    x = x / 5
    x = x* 10
    x = int(x)
    return x

rat_udf=udf(rating_conv,IntegerType())

new_DF = df.withColumn('rating',rat_udf(df.rating))
new_DF.filter("titleID is NULL").show()


In [79]:
new_DF.summary

<bound method DataFrame.summary of DataFrame[userID: string, titleID: int, rating: int, r_type: string]>

In [78]:
new_DF \
   .repartition(1) \
   .write.format("com.databricks.spark.csv") \
   .option("header", "true")  \
   .save("/root/datasets/FYP/O/agg/ratings")


In [2]:
#making a movie reference file

#Loading reference file and storing it as a python dict obj
with open('/root/datasets/ml-20m/links.csv', mode='r') as infile:
    reader = csv.reader(infile)
    with open('coors_new.csv', mode='w') as outfile:
        writer = csv.writer(outfile)
        mydict = {rows[0]:rows[2] for rows in reader}

In [3]:
df_movies=spark.read.csv('/root/datasets/ml-20m/movies.csv',inferSchema=True,header=True)
#Converting titleIDs to TMDB Ids by writing UDF and searching in lookup dict
def get_mov(x):
    try:
        return mydict[x]
    except:
        return '0'

get=udf(get_mov,StringType())
df_movies = df_movies.withColumn("movieId", df_movies["movieId"].cast("string"))
df_movies = df_movies.withColumn('movieId',get(df_movies.movieId))
df_movies = df_movies.withColumn("movieId", df_movies["movieId"].cast("integer"))
df_movies = df_movies.withColumnRenamed("movieId", "titleID")


In [86]:

df_movies.write.format('csv').option('header',True).option('sep',',').save('/root/datasets/FYP/O/agg/movies')


In [4]:
df_movies.show()

+-------+--------------------+--------------------+
|titleID|               title|              genres|
+-------+--------------------+--------------------+
|    862|    Toy Story (1995)|Adventure|Animati...|
|   8844|      Jumanji (1995)|Adventure|Childre...|
|  15602|Grumpier Old Men ...|      Comedy|Romance|
|  31357|Waiting to Exhale...|Comedy|Drama|Romance|
|  11862|Father of the Bri...|              Comedy|
|    949|         Heat (1995)|Action|Crime|Thri...|
|  11860|      Sabrina (1995)|      Comedy|Romance|
|  45325| Tom and Huck (1995)|  Adventure|Children|
|   9091| Sudden Death (1995)|              Action|
|    710|    GoldenEye (1995)|Action|Adventure|...|
|   9087|American Presiden...|Comedy|Drama|Romance|
|  12110|Dracula: Dead and...|       Comedy|Horror|
|  21032|        Balto (1995)|Adventure|Animati...|
|  10858|        Nixon (1995)|               Drama|
|   1408|Cutthroat Island ...|Action|Adventure|...|
|    524|       Casino (1995)|         Crime|Drama|
|   4584|Sen

In [81]:

df_movies=spark.read.csv('/root/datasets/FYP/O/agg/ratings/orgrating.csv',inferSchema=True,header=True)


In [82]:
df_movies.summary

<bound method DataFrame.summary of DataFrame[userID: string, titleID: int, rating: int, r_type: string]>

In [2]:
spark.stop()